In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyreadstat
import os

In [2]:
pd.set_option('display.max_columns', None)
os.chdir('C:/Users/511232/Desktop/MICS/microdata')
[f for f in os.listdir() if 'sav' in f]

['bh.sav', 'ch.sav', 'fs.sav', 'hh.sav', 'hl.sav', 'wm.sav']

In [55]:
os.chdir('C:/Users/511232/Desktop/MICS/microdata')
df_hh,meta_hh=pyreadstat.read_sav('hh.sav', apply_value_formats=False)
df_wm,meta_wm=pyreadstat.read_sav('wm.sav', apply_value_formats=False)
df_hl,meta_hl=pyreadstat.read_sav('hl.sav', apply_value_formats=False)

col_names_hh=meta_hh.column_names_to_labels
col_vals_hh=meta_hh.variable_value_labels
col_names_hl=meta_hl.column_names_to_labels
col_vals_hl=meta_hl.variable_value_labels
col_names_wm=meta_wm.column_names_to_labels
col_vals_wm=meta_wm.variable_value_labels

data_hh=df_hh.copy()
data_wm=df_wm.copy()
data_hl=df_hl.copy()

#############

cond_smoker_missing=((data_wm['TA1']==9) | (data_wm['TA3']==9) | (data_wm['TA5']==99)) | \
((data_wm['TA6']==9) | (data_wm['TA7']==9) | (data_wm['TA9']==99)) | \
    ((data_wm['TA10']==9) | (data_wm['TA11']==9) | (data_wm['TA13']==99))

cond_smoker=(((data_wm['TA5']>0) & (data_wm['TA5']<99)) | \
((data_wm['TA9']>0) & (data_wm['TA9']<99)) | \
    ((data_wm['TA13']>0) & (data_wm['TA13']<99)))

cond_non_smoker=(((data_wm['TA1']==2) | (data_wm['TA2']==0) | (data_wm['TA3']==2) | (data_wm['TA5']==0)) & \
((data_wm['TA6']==2) | (data_wm['TA7']==2) | (data_wm['TA9']==0)) & \
    ((data_wm['TA10']==2) | (data_wm['TA11']==2) | (data_wm['TA13']==0)))

data_wm['Smoker']=np.select([cond_smoker_missing,cond_smoker,cond_non_smoker], 
['Missing','Smoker','Non-smoker'], default=np.nan)

#create literacy
'''literate (So if WB6A = 2 | WB6A = 3 | (WB6A = 1 & WB6B >=5) --> Literate)'''
cond=[(data_wm['WB14']==3) | (data_wm['WB6A']==2) | (data_wm['WB6A']==3) | ((data_wm['WB6A']==1) & (data_wm['WB6B']>=5)),
data_wm['WB14'].isna(),data_wm['WB14']==9]
rslt=['Literate',np.nan,'No response']
data_wm['Literacy']=np.select(cond,rslt,default='Illiterate')

#create edu_level  (!!!!WARNING!!! it only applies for PALESTINE)
edu_lev_label={1:'ISCED X',2:'ISCED 0',3:'ISCED 1',4:'ISCED 2',5:'ISCED 3',6:'ISCED 4',7:'ISCED 5',
8:'ISCED 6',9:'ISCED 7',10:'ISCED 8',11:'Not classifiable', 12:'Not stated'}

cond=[data_wm['WB5']==2,data_wm['WB5']==9,
data_wm['WB6A']==0,data_wm['WB6A']==1,
data_wm['WB6A']==2,data_wm['WB6A']==3]

rslt=[1,12,2,3,5,8]

data_wm['edu_level']=np.select(cond,rslt)
data_wm['edu_level']=data_wm['edu_level'].map(edu_lev_label)

#create Birth_Skilled_Per 'Birth attended by skilled personnels'
'''IF (MN19A = "?" | MN19B = "?" | MN19H = "?" | MN19X = "?" | MN19Y = "?" | MN19NR = "?") Birth_Skilled_Per = 9.
IF (MN19H = "H" | MN19X = "X" | MN19Y = "Y") Birth_Skilled_Per = 2.
IF (MN19A = "A" | MN19B = "B") Birth_Skilled_Per = 1.
'''
Birth_Skilled_Per_label={1:'Yes', 2:'No', 9:'No repsonse'}
cond=[(data_wm['MN19A']=='A') | (data_wm['MN19B']=='B'),
(data_wm['MN19H']=='H') | (data_wm['MN19X']=='X') | (data_wm['MN19Y']=='Y'),
(data_wm['MN19NR']=='?')]
rslt=[1, 2,9]

data_wm['Birth_Skilled_Per']=np.select(cond,rslt,default=np.nan)
data_wm['Birth_Skilled_Per']=data_wm['Birth_Skilled_Per'].map(Birth_Skilled_Per_label)

#agegrp5[<15,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65+]
cond=[(data_hl['HL6']>=15)&(data_hl['HL6']<=19),
(data_hl['HL6']>=20)&(data_hl['HL6']<=24),
(data_hl['HL6']>=25)&(data_hl['HL6']<=29),
(data_hl['HL6']>=30)&(data_hl['HL6']<=34),
(data_hl['HL6']>=35)&(data_hl['HL6']<=39),
(data_hl['HL6']>=40)&(data_hl['HL6']<=44),
(data_hl['HL6']>=45)&(data_hl['HL6']<=49),
(data_hl['HL6']>=50)&(data_hl['HL6']<=54),
(data_hl['HL6']>=55)&(data_hl['HL6']<=59),
(data_hl['HL6']>=60)&(data_hl['HL6']<=64),
data_hl['HL6']>=65,data_hl['HL6'].isna()]
result=['15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65+','Missing']
data_hl['agegrp5']=np.select(cond,result,default='<15')


#create hh_size
data_hh['hh_size']=np.where(data_hh['HH48']>=8, '8+',data_hh['HH48'])
#create living_alone
cond=[data_hh['HH48']==1,data_hh['HH48'].isna(), data_hh['HH48']>1]
result=['alone','Missing','not alone']
data_hh['living_alone']=np.select(cond,result)

#create modern method of contraception 'modern_contraceptive'
'''
'CP4A': {'?': 'NO RESPONSE', 'A': 'FEMALE STERILIZATION'},
'CP4B': {'?': 'NO RESPONSE', 'B': 'MALE STERILIZATION'},
'CP4C': {'?': 'NO RESPONSE', 'C': 'IUD'},
'CP4D': {'?': 'NO RESPONSE', 'D': 'INJECTABLES'},
'CP4E': {'?': 'NO RESPONSE', 'E': 'IMPLANTS'},
'CP4F': {'?': 'NO RESPONSE', 'F': 'PILL'},
'CP4G': {'?': 'NO RESPONSE', 'G': 'MALE CONDOM'},
'CP4H': {'?': 'NO RESPONSE', 'H': 'FEMALE CONDOM'},
'CP4I': {'?': 'NO RESPONSE', 'I': 'DIAPHRAGM'},
'CP4J': {'?': 'NO RESPONSE', 'J': 'FOAM / JELLY'}'''

methods=['A','B','C','D','E','F','G','H','I','J']
cols=['CP4A','CP4B','CP4C','CP4D','CP4E','CP4F','CP4G','CP4H','CP4I','CP4J']
data_wm['modern_contraceptive']=data_wm[cols].apply(lambda x: int(any(x.isin(methods))), axis=1)
modern_contraceptive_labels={0:'No', 1:'Yes'}
data_wm['modern_contraceptive']=data_wm['modern_contraceptive'].map(modern_contraceptive_labels)

#MERGE [HH ---> WM] on [HH1,HH2] to add 'HC14': 'Household owns the dwelling'
right_df=data_hh[['HH1','HH2','HC14','HC0','HC3','WS1','EU1','WS5','WS7','WS9','WS11','HW1']]
left_df=data_wm
data_wm=pd.merge(left_df,right_df, how='left',on=['HH1','HH2'])

#MERGE [HH ---> HL] to get 'hh_size','living_alone'
right_df=data_hh[['HH1','HH2','hh_size','living_alone']]
left_df=data_hl
df_hl=pd.merge(left_df,right_df, how='left',on=['HH1','HH2'])

#MERGE [HL ---> WM] to add 'hh_size','HL3'(household head relation),'HL6'(age),'hh_type'
right_df=df_hl[['HH1','HH2','HL1','hh_size','HL3','HL6','living_alone','agegrp5']]
left_df=data_wm
data_wm=pd.merge(left_df,right_df, how='left', 
left_on=['HH1','HH2','LN'], right_on=['HH1','HH2','HL1'])

#create head of household relationship variable as 1:HH 2:Other 
data_wm['hh_rel']=np.where(data_wm['HL3']==1,1,2)
data_wm['hh_rel']=data_wm['hh_rel'].map({1:'Head of household', 2:'Other'})

#recode NA in disability to 9
data_wm['disability'].fillna('9', inplace = True)

#get rid of 329 NAs across almost all columns
data_wm_filtered=data_wm.dropna(axis=1, how='any')
print(f'{data_wm.shape[0] - data_wm_filtered.shape[0]} rows are dropped')


0 rows are dropped


In [57]:
data_wm_filtered.shape

(11464, 128)

In [50]:
col_used=['WB18','MT1','MT2','MT3','MT11','MA1','VT21','VT1','VT9',\
    'windex5','disability','Smoker','Literacy','living_alone','edu_level',\
        'agegrp5','hh_size','modern_contraceptive',\
            'HC0','HC3','WS1','EU1','WS5','WS7','WS9','WS11','HW1']
df=data_wm_filtered[col_used]

print(df.shape[0])

11464


In [52]:
col_wm_dict=col_names_wm
col_hh_dict=col_names_hh

col_union=list(set(col_wm_dict.keys()).union(set(col_hh_dict.keys())))
col_int=set(col_union).intersection(col_used)

def merge_dict(col):
    if col in col_wm_dict.keys():
        return(col_wm_dict[col])
    else:
        return(col_hh_dict[col])

col_dict={col:merge_dict(col) for col in col_int}

In [53]:
df.rename(columns=col_dict, inplace=True)

c:\Users\511232\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [54]:
col=df.isna().sum()[df.isna().sum()>0].index

for c in col:
    print(df[c].value_counts(dropna=False),'\n')

1.0    8684
2.0    2443
NaN     329
9.0       8
Name: Covered by health insurance, dtype: int64 

0.0    7084
1.0    2082
2.0    1180
3.0     771
NaN     329
9.0      18
Name: Frequency of reading newspaper or magazine, dtype: int64 

0.0    7420
1.0    1830
2.0     940
3.0     931
NaN     329
9.0      14
Name: Frequency of listening to the radio, dtype: int64 

3.0    5542
2.0    2137
0.0    1738
1.0    1652
NaN     329
9.0      66
Name: Frequency of watching TV, dtype: int64 

1.0    7455
NaN    2708
2.0    1294
9.0       7
Name: Ever used internet, dtype: int64 

1.0    9602
2.0    1523
NaN     329
9.0      10
Name: Own a mobile phone, dtype: int64 

1.0    6880
3.0    4255
NaN     329
Name: Currently married or living with a man, dtype: int64 

2.0    5921
1.0    3513
3.0    1098
7.0     447
NaN     329
4.0     150
9.0       6
Name: Feeling safe at home alone after dark, dtype: int64 

2.0    10824
NaN      329
1.0      291
8.0       12
9.0        8
Name: Someone taken or tried tak